Note: This notebook is derived from https://raw.githubusercontent.com/pytorch/examples/master/mnist/main.py This work is licensed using PyTorch licensing terms here: https://github.com/pytorch/pytorch/blob/master/LICENSE 

In [17]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
batch_size=64
test_batch_size=1000
epochs=2
lr=0.01
momentum=0.5
no_cuda=False
seed=1
log_interval=10

In [4]:
torch.manual_seed(seed)

In [5]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size, shuffle=True, **kwargs)

Processing...
Done!


In [7]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    test_batch_size, shuffle=True, **kwargs)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model = Net().to(device)

In [10]:
optimizer = optim.SGD(model.parameters(), lr, momentum)

In [14]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [15]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.331304
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.286626
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.286227
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.283814
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.262959
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.264584
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.199159
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.170336
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.122321
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.008252
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.983888
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.901633
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.802673
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.726889
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.551356
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.495289
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.273458
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.304586
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.184681
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.104361
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.289848
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.358061
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.401612
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.596988
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.330238
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.217486
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.428951
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.399002
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.301578
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.331483
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.305854
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.303773
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.210863
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.221762
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.290449
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.290034
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.433287
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.495906
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.339379
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.278974


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.181867
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.408920
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.340544
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.294808
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.161962
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.158374
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.236017
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.308894
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.505486
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.208319
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.126996
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.327542
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.231161
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.127788
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.173151
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.210434
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.248471
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.195961
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.347178
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.292329


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.203668
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.202742
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.172945
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.261535
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.127217
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.404167
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.167105
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.361388
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.327771
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.195493
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.185830
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.254970
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.142848
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.257694
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.187112
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.211735
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.192357
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.140778
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.237728
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.188352


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.105711
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.176102
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.295593
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.062641

Test set: Average loss: 0.0657, Accuracy: 9799/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.104702
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.323023
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.524758
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.235531
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.223226
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.114681
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.276168
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.158431
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.174304
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.185460
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.242995
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.109909
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.080497
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.121178
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.228485
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.164102
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.141664
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.211169
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.193981
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.102826
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.168156
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.188780
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.181349
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.084832
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.242199
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.114843
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.087005
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.313981
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.311594
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.058887
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.245970
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.174618
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.236051
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.126266
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.241889
